In [1]:
import numpy as np
import pandas as pd
import time
import sys
sys.path.append(r"C:\Program Files\IBM\ILOG\CPLEX_Studio2211\cplex\python\3.9\x64_win64")
import cplex
!pip install docplex
import docplex.mp.model as mip

You should consider upgrading via the 'C:\Users\mukun\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [ ]:
df = pd.read_csv(r"C:\Users\mukun\Downloads\1year_seed0_instance_simulation.csv")

df

,Week,Day,Weekday,Event Type,Next Event Type,Ticket Price,Attendance,Overhead,Setup Costs,Cleanup Costs,Special Event
0,1,1,1,0,0,277.436687,18148,62824.41,4179.401345,6595.471164,0
1,1,1,1,1,0,259.140256,14552,62824.41,5118.857079,8097.889303,0
2,1,1,1,2,0,333.050808,18245,62824.41,3755.108419,11053.826053,0
3,1,1,1,3,0,74.633053,20210,62824.41,6686.009836,10304.576606,0
4,1,1,1,4,0,81.805099,22325,62824.41,3689.975886,8417.901652,1
...,...,...,...,...,...,...,...,...,...,...,...
1820,53,365,1,0,0,277.972339,19052,62824.41,3846.809084,6328.627384,0
1821,53,365,1,1,0,256.108227,16530,62824.41,5959.528831,7558.309887,0
1822,53,365,1,2,0,146.688019,5058,62824.41,4064.871761,10941.200893,0
1823,53,365,1,3,0,74.458078,21385,62824.41,4798.295738,11023.393936,0


In [34]:
df = df.drop(['Weekday', 'Next Event Type'], axis = 1)
df['Day'] = df['Day'] -1
df['Week'] = df['Week'] -1
df2 = df.groupby(['Day', 'Event Type']).mean()
n = df2.Attendance
s = df2['Ticket Price'].abs()
SC = df2['Setup Costs']
CC = df2['Cleanup Costs']
OC = df2.Overhead


In [35]:
special_events = df2['Special Event']
special_events = special_events[special_events == 1]
special_events
special_event_days = []


for i in range(len(special_events)):
  special_event_days.append(special_events.index[i][0])
special_event_days
special_event_days = special_event_days #for now
special_event_days

[0, 26, 36, 53, 62, 64, 104, 109, 146, 200, 216, 218, 291, 311, 328, 338, 361]

In [36]:
events = [0,1,2,3,4]
sports_events = [0,1]
days = []
for i in range(364):
  days.append(i)
in_season = []
for i in range(231):
  in_season.append(i)
off_season = []
for i in range(231,364):
  off_season.append(i)

weeks = []
for i in range(52):
  weeks.append(i)

weekset = []
weekday = []
weeknumber = 0
for i in weeks:
  for j in range(7):
    weekday.append(weeknumber)
    weeknumber += 1
  weekset.append(weekday)
  weekday = []

In [37]:
df2

Week  Ticket Price  Attendance  Overhead  Setup Costs   
Day Event Type                                                          
0   0            0.0    277.436687     18148.0  62824.41  4179.401345  \
    1            0.0    259.140256     14552.0  62824.41  5118.857079   
    2            0.0    333.050808     18245.0  62824.41  3755.108419   
    3            0.0     74.633053     20210.0  62824.41  6686.009836   
    4            0.0     81.805099     22325.0  62824.41  3689.975886   
...              ...           ...         ...       ...          ...   
364 0           52.0    277.972339     19052.0  62824.41  3846.809084   
    1           52.0    256.108227     16530.0  62824.41  5959.528831   
    2           52.0    146.688019      5058.0  62824.41  4064.871761   
    3           52.0     74.458078     21385.0  62824.41  4798.295738   
    4           52.0     82.132680     22325.0  62824.41  3972.218514   

                Cleanup Costs  Special Event  
Day Event Type                                
0   0             6595.471164            0.0  
    1             8097.889303            0.0  
    2            11053.826053            0.0  
    3            10304.576606            0.0  
    4             8417.901652            1.0  
...                       ...            ...  
364 0             6328.627384            0.0  
    1             7558.309887            0.0  
    2            10941.200893            0.0  
    3            11023.393936            0.0  
    4             9859.439983            0.0  

[1825 rows x 7 columns]

In [ ]:
#@title MIP Model Completed

#Model
mip_md1 = mip.Model("MIP")
#Variable creation
x = mip_md1.integer_var_dict(keys = ((i,j) for i in days for j in events), lb = 0, ub = 1, name = 'x')
z = mip_md1.integer_var_dict(keys = ((m,n) for m in in_season for n in sports_events), lb = 0, ub = 1, name = 'z')

mip_md1.parameters.mip.tolerances.mipgap.set(1e-6) 
mip_md1.context.solver.log_output = True

#Constraints

#At most, only one event can occur per day, although the venue can also have no events scheduled for a day as well
for i in days:
  mip_md1.add_constraint(mip_md1.sum(x[i,j] for j in events) <= 1) #yes

#Venue’s home basketball team must play exactly 41 games at the venue. A typical NBA team plays 82 games per season, of which it is expected that half will be at the home venue
mip_md1.add_constraint(mip_md1.sum(x[t,0] for t in in_season) == 41) #yes

#Similarly, the venue’s home hockey team must play exactly 41 games at the venue, as a typical NHL team plays 82 games per season
mip_md1.add_constraint(mip_md1.sum(x[t,1] for t in in_season) == 41) #yes

#There must be no games scheduled in the sports offseason, that is, between days 212-365.
for i in sports_events:
  mip_md1.add_constraint(mip_md1.sum(x[t,i] for t in off_season) == 0) #yes

#There cannot be three days in a row where the same sporting event is played.
for i in sports_events:
  for t in range(len(in_season)-2):
    mip_md1.add_constraint(mip_md1.sum(x[k,i] for k in range(t,t+3)) <= 2) #yes

#The total number of days where the venue is in use must be less than 300.
mip_md1.add_constraint(mip_md1.sum(x[i,j] for i in days for j in events) <= 300) #yes

#The total number of days where the venue is in use must be at least 100.
mip_md1.add_constraint(mip_md1.sum(x[i,j] for i in days for j in events) >= 100) #yes

#There can be up to 4 events scheduled within a 7 day period, but no more.
for k in weekset:
  mip_md1.add_constraint(mip_md1.sum(x[w,i] for w in k for i in events) <= 4) #yes

#There cannot be more than 3 sporting events scheduled in a 7 day period.
for k in weekset:
  mip_md1.add_constraint(mip_md1.sum(x[w,i] for w in k for i in sports_events) <= 3) #yes
 
#At least one concert must be scheduled in a given 7-day period.
for k in weekset:
  mip_md1.add_constraint(mip_md1.sum(x[w,2] for w in k) >= 1) #yes

#There have to be at least 15 exhibitions/conventions scheduled a year.
mip_md1.add_constraint(mip_md1.sum(x[i,3] for i in days) >= 15) #yes


# There must be 1 day of lag after a concert to account for takedown/cleanup time.
for t in range(len(days) - 1):
    for i in events:
        mip_md1.add_constraint(x[t, 2] + x[t + 1, i] <= 1) #yes

# There must be 1 day of lag before a concert to account for setup time.
for t in range(1, len(days)):
    for i in events:
        mip_md1.add_constraint(x[t - 1, i] + x[t, 2] <= 1) #yes

#Back to back basketball games, and back to back hockey games may only occur up to 8 times for each sport, but no more.
for i in sports_events:
  for j in range(len(in_season) - 1):

    mip_md1.add_constraint(z[j,i] <= x[j,i])
    mip_md1.add_constraint(z[j,i] <= x[j+1, i])
    mip_md1.add_constraint(z[j,i] >= x[j,i] + x[j+1, i] - 1)

for i in sports_events:
  mip_md1.add_constraint(mip_md1.sum(z[t,i] for t in range(len(in_season)-1)) <= 8) #yes

for t in special_event_days:
  mip_md1.add_constraint(x[t,4] == 1)


mip_md1.maximize(mip_md1.sum(s.loc[(t,i)] * x[t,i] - SC.loc[(t,i)]*x[t,i] - CC.loc[(t,i)]*x[t,i] - OC.loc[(t,i)]*x[t,i] for t in days for i in events))


#initial_vars = mip_md1.number_of_variables
#print(initial_vars)
#num_constraints_before_presolve = mip_md1.number_of_constraints
# Solve the model
mip_md1.parameters.mip.strategy.search = 1
soln = mip_md1.solve(log_output=False)
with open("solver_log.txt", "w") as log_file:
    mip_md1.solve(log_output=log_file)


with open("solver_log_365_instance_1.txt", "w") as log_file:

    mip_md1.solve(log_output=log_file)

    if soln is not None:
        log_file.write(f"Objective Value: {mip_md1.objective_value}\n")
        if soln is not None:
            original_stdout = sys.stdout
            sys.stdout = log_file  
            mip_md1.print_solution()  
            sys.stdout = original_stdout 
        else:
            print("* Model is infeasible")

if soln is not None:
  total_ones_in_x = sum(value for (i, j), value in mip_md1.solution.get_value_dict(x).items() if value == 1)
  print({total_ones_in_x})
  mip_md1.print_solution()

else:
  print("* model is infeasible")

stats = mip_md1.get_statistics()




Total number of 1s in x variables: 166.0
objective: -12496860.702
status: OPTIMAL_SOLUTION(2)
  x_0_4=1
  x_2_2=1
  x_4_0=1
  x_6_1=1
  x_8_0=1
  x_9_0=1
  x_10_3=1
  x_12_2=1
  x_15_2=1
  x_17_0=1
  x_18_1=1
  x_19_0=1
  x_21_2=1
  x_23_3=1
  x_24_1=1
  x_26_4=1
  x_28_2=1
  x_31_1=1
  x_33_0=1
  x_34_3=1
  x_35_1=1
  x_36_4=1
  x_38_1=1
  x_40_2=1
  x_43_3=1
  x_44_0=1
  x_46_2=1
  x_48_0=1
  x_50_0=1
  x_52_1=1
  x_53_4=1
  x_55_2=1
  x_57_1=1
  x_58_1=1
  x_60_2=1
  x_62_4=1
  x_64_4=1
  x_65_1=1
  x_67_2=1
  x_69_1=1
  x_70_1=1
  x_73_2=1
  x_75_1=1
  x_76_1=1
  x_77_0=1
  x_78_1=1
  x_81_2=1
  x_83_0=1
  x_84_1=1
  x_85_1=1
  x_87_1=1
  x_89_2=1
  x_91_2=1
  x_94_1=1
  x_95_0=1
  x_96_1=1
  x_98_2=1
  x_100_0=1
  x_103_1=1
  x_104_4=1
  x_105_0=1
  x_107_2=1
  x_109_4=1
  x_111_0=1
  x_113_0=1
  x_114_0=1
  x_116_2=1
  x_119_2=1
  x_121_0=1
  x_122_1=1
  x_125_0=1
  x_126_1=1
  x_128_2=1
  x_130_0=1
  x_131_0=1
  x_133_2=1
  x_136_1=1
  x_137_0=1
  x_139_1=1
  x_141_2=1
  x_144_0

In [14]:
months = ['Oct','Nov','Dec','Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sept']
months_of_year = dict.fromkeys(months, [])
months_of_year_range = [(0,31), (31,61),(61,92),(92,122),(122,153),(153,181),(181,212),(212,242),(242,273),(273,304),(304,335),(335,364)]
zipped = zip(months, months_of_year_range)
month_range = dict.fromkeys(months,[])
for i, j in zipped:
  month_range[i] = j
month_range

{'Oct': (0, 31),
 'Nov': (31, 61),
 'Dec': (61, 92),
 'Jan': (92, 122),
 'Feb': (122, 153),
 'Mar': (153, 181),
 'Apr': (181, 212),
 'May': (212, 242),
 'Jun': (242, 273),
 'Jul': (273, 304),
 'Aug': (304, 335),
 'Sept': (335, 364)}

In [15]:
for mth, days in month_range.items():
  month_list = []
  for i in x:
    if i[0] in range(days[0],days[1]):
      value = mip_md1.solution.get_value(x[i])
      if value == 1:
        month_list.append(i)
  months_of_year[mth] = month_list
print(months_of_year)

{'Oct': [(1, 2), (3, 1), (5, 1), (6, 0), (7, 1), (8, 1), (10, 2), (12, 0), (14, 2), (17, 1), (18, 0), (20, 2), (22, 4), (24, 2), (26, 0), (27, 1), (29, 0), (30, 4)], 'Nov': [(32, 4), (34, 2), (37, 2), (39, 0), (40, 0), (41, 1), (42, 4), (44, 2), (46, 1), (47, 1), (49, 2), (51, 2), (53, 0), (54, 0), (56, 0), (57, 1), (58, 1), (60, 2)], 'Dec': [(63, 2), (66, 2), (68, 0), (69, 0), (71, 0), (72, 4), (74, 2), (76, 0), (78, 2), (80, 0), (82, 0), (83, 1), (85, 4), (86, 1), (87, 4), (89, 2), (91, 0)], 'Jan': [(93, 1), (95, 2), (97, 0), (99, 2), (102, 1), (103, 0), (104, 1), (105, 1), (108, 1), (109, 1), (111, 2), (114, 2), (116, 1), (117, 4), (118, 0), (119, 1), (120, 1)], 'Feb': [(122, 2), (124, 4), (126, 2), (129, 1), (130, 0), (131, 0), (133, 2), (135, 1), (137, 1), (139, 2), (141, 2), (143, 1), (145, 0), (146, 0), (147, 1), (148, 1), (149, 0), (151, 2)], 'Mar': [(154, 0), (155, 4), (157, 2), (160, 1), (161, 0), (163, 1), (164, 0), (166, 2), (168, 0), (169, 1), (172, 2), (174, 1), (175, 0),

In [121]:
for i in months_of_year:
  print(len(months_of_year[i]))

18
17
17
17
19
15
19
16
18
18
17
16


In [122]:
flattened_data = [(month, day, value) for month, days in months_of_year.items() for day, value in days]
df = pd.DataFrame(flattened_data, columns=['Month', 'Day', 'Value'])

# Save to CSV
df.to_csv('instance_0_scheduele.csv', index=False)

In [20]:
import os
os.getcwd()

'c:\\Users\\mukun'